# To dos:
1. Make nice output. D3 and javascript?
- - - 

In [1]:
import pandas as pd
import os
%matplotlib inline

In [2]:
def split_col(row):
    try:
        row_col = row.Row[1:]
    except TypeError:
        row_col = 'NaN'
    else:
        return row_col
    
def split_row(row):
    try:
        row_Row = row.Row[0]
    except TypeError:
        row_Row = 'NaN'
    else:
        return row_Row

In [3]:
# dictionary to reverse transcribe sequences
rev_xcr = {'a':'t', 'c':'g', 'g':'c', 't':'a'}

# read in files from lab members

# Candice
df = pd.read_csv('Candice.csv')
df.columns = ['Date', 'Box', 'Row', 'Col', 'Name', 'Sequence', 'Misc', 'Other']
df['LabMember'] = 'Candice Yip'
df.Date = df.Date.fillna(method='ffill')

# Standard promoters from Eton
temp_std = pd.read_excel('StandardPrimers.xlsx')
temp_std['LabMember'] = 'Eton Primer'
temp_std['Box'] = 'NaN'
temp_std['Row'] = 'NaN'
temp_std['Col'] = 'NaN'
temp_std['Misc'] = 'NaN'
temp_std['Other'] = 'NaN'
temp_std['Date'] = 'NaN'
temp_std = temp_std[['Date', 'Box', 'Row', 'Col', 'Name', 'Sequence', 'Misc', 'Other', 'LabMember']]

# Elizabeth
temp_elizabeth = pd.read_excel('Elizabeth.xlsx')
temp_elizabeth.columns = ['Name', 'Sequence', 'Box', 'Row', 'Col', 'Misc', 'Other']
temp_elizabeth['LabMember'] = 'Elizabeth Lamkin'
temp_elizabeth['Date'] = 'NaN'
temp_elizabeth = temp_elizabeth[['Date', 'Box', 'Row', 'Col', 'Name', 'Sequence', 'Misc', 'Other', 'LabMember']]

# Karolina
temp_Kar = pd.read_excel('Karolina.xlsx', sheetname = 'KM primers')
temp_Kar['Other'] = temp_Kar['Unnamed: 9']
temp_Kar.columns = ['Box', 'Row', 'Name', 'Sequence', 'Date', 'Misc', 'U6', 'U7', 'U8', 'U9', 'U10', 'Other']
temp_Kar['LabMember'] = 'Karolina Mizeracka'
temp_Kar['Col'] = temp_Kar.apply(split_col, axis=1)
temp_Kar.Row = temp_Kar.apply(split_row, axis=1)
temp_Kar = temp_Kar[['Date', 'Box', 'Row', 'Col', 'Name', 'Sequence', 'Misc', 'Other', 'LabMember']]

# Max
temp_Max = pd.read_excel('Max.xls')
temp_Max.columns = ['OldBox', 'Box', 'Name', 'Sequence', 'Row', 'Col', 'Date', 'Misc']
temp_Max['LabMember'] = 'Max Heiman'
temp_Max['Other'] = 'NaN'
temp_Max = temp_Max[['Date', 'Box', 'Row', 'Col', 'Name', 'Sequence', 'Misc', 'Other', 'LabMember']]

In [4]:
df = df.append(temp_std, ignore_index=True)
df = df.append(temp_elizabeth, ignore_index=True)
df = df.append(temp_Max, ignore_index=True)
df = df.append(temp_Kar, ignore_index=True)

In [5]:
for file in os.listdir():
    if file not in ['.DS_Store', '.ipynb_checkpoints', 'Candice.csv', 'Elizabeth.xlsx', 'Karolina.xlsx', 'Max.xls','Primers search.ipynb', 'StandardPrimers.xlsx']:
        try:
            temp = pd.read_excel(file)
            try:
                temp.columns = ['Name', 'Sequence', 'Date', 'Box', 'Row', 'Misc']
                temp['Other'] = 'NaN'
            except ValueError:
                temp.columns = [['Name', 'Sequence', 'Date', 'Box', 'Row', 'Misc', 'Other']]
            temp['LabMember'] = file.split('.')[0]
            temp['Col'] = temp.apply(split_col, axis=1)
            temp.Row = temp.apply(split_row, axis=1)        
            temp = temp[['Date', 'Box', 'Row', 'Col', 'Name', 'Sequence', 'Misc', 'Other', 'LabMember']]
            df = df.append(temp, ignore_index=True)
        except:
            print ('Could not read', file)
            pass

In [6]:
print ('Number of primers by lab member: ')
print (df.groupby('LabMember').Name.count())

Number of primers by lab member: 
LabMember
Candice Yip           390
Claire Williams       171
Elizabeth Lamkin      351
Eton Primer            23
Isabel Low            116
Karolina Mizeracka    385
Max Heiman            712
Megan Chong            98
Name: Name, dtype: int64


# Clean sequence input from user

Clean up input sequence from user.

In [7]:
print ('Paste your sequence to find primers. Uppercase or lowercase, spaces, and line breaks allowed.')
sequence = input('You can input more than one sequence, just separate sequences with a semicolon: ')
print ()

# test sequence
# sequence = 'tagttcatccatgccaaaa;   actgactga tg aatttaactttgacgg ACGTACGT \n ; aaaaagggggcgatactggtactaa; acgtacgatgcatgcatgcatgctga atttaactttgacgg aaacgctactgacgtacgatacgatctgactga agtatctcatgactg'
# sequence = 'acgtacgatgcatgcatgcatgctga atttaactttgacgg aaacgctactgacgtacgatacgatctgactga agtatctcatgactg'
# sequence = 'tgtaaaacgacggccagtgaatt'
# for_sequencing = input("Are you looking for sequencing primers for these sequences? y/n: ")

# split if multiple sequences
sequence_list = sequence.split(sep=';')

# clean up sequence(s)
temp_list = []
for sequence in sequence_list:
    sequence = sequence.lower().replace(" ", "").strip(' \t\n\r')
    for base in sequence:
        if base not in ['a', 'g', 'c', 't']:
            print ('Error. Your sequence contains letters other than base pairs (a, g, c, t)')
            break
    temp_list.append(sequence)

sequence_list = temp_list

Paste your sequence to find primers. Uppercase or lowercase, spaces, and line breaks allowed.
You can input more than one sequence, just separate sequences with a semicolon: agct



# Clean primers dataframe

Clean primer sequences.

In [8]:
def clean_primer(primer):
    try:
        primer = primer.lower().replace(" ", "").strip()
        for base in primer:
            if base not in ['a', 'g', 'c', 't']:
                print ('Warning: One or more of your primers had stuff that were not dna bases. I replaced them with null values.')
                primer = 'NaN'
                break
    except AttributeError:
        primer = 'NaN'
    return primer
    
df.Sequence = df['Sequence'].apply(clean_primer)

Clean box and column floats.

In [9]:
def integerize_locations(box):
    try:
        return str(int(box))
    except:
        return 'NaN'
df['Box'] = df['Box'].apply(integerize_locations)
df['Col'] = df['Col'].apply(integerize_locations)

Get forward and reverse primer segments (15 nucleotides from 3' end), which we will use to match input sequence.

In [10]:
def forward_15(primer):
    try:
        primer_15 = primer[-15:]
    except:
        primer_15 = 'NaN'
    return primer_15
def reverse_15(primer):
    try:
        primer_rev = ""    

        # reverse transcribe
        for i in primer:
            primer_rev += rev_xcr[i]
        
        # flip order
        primer_rev = primer_rev[::-1]
        
        # grab first 15 nucleotides
        primer_rev_15 = primer_rev[:15]
        
    except:
        primer_rev_15 = 'NaN'
    return primer_rev_15

df['Forward_15'] = df['Sequence'].apply(forward_15)
df['Reverse_15'] = df['Sequence'].apply(reverse_15)

Locate forward and reverse primers in sequence.

In [11]:
def locateFprimer(row):
    locate_box = {}

    try:
        if type(row.Forward_15) == str and sequence.find(row.Forward_15) >= 0:
            locate_box[sequence.find(row.Forward_15)] = ['Name: ' + row.Name,
                                                         'Fwd',
                                                         'Box ' + row.Box + row.Row + row.Col,
                                                         'Entire primer in sequence? ' + str(sequence.find(row.Sequence) != -1),
                                                         'Primer: ' + row.Sequence,
                                                         row.LabMember,
                                                         'Notes: '+ str(row.Misc) + '. Misc: ' + str(row.Other),
                                                         'Date: ' + row.Date
                                                        ]
    except:
        pass
    
    if len(locate_box.keys()) > 0:
        return (locate_box)

    
    
def locateRprimer(row):
    locate_box = {}
    
    
    try:
        rev_seq = ""
        for base in row.Sequence:
            rev_seq += rev_xcr[base]
        rev_seq = rev_seq[::-1]
    except:
        pass
    
    try:
        if type(row.Reverse_15) == str and sequence.find(row.Reverse_15) >= 0:
            locate_box[sequence.find(row.Reverse_15)] = ['Name: ' + row.Name, 
                                                         'Rev', 
                                                         'Box ' + row.Box + row.Row + row.Col,
                                                         'Entire primer in sequence? ' + str(sequence.find(rev_seq) != -1),
                                                         'Primer: ' + row.Sequence,
                                                         row.LabMember,
                                                         'Notes: '+ str(row.Misc) + '. Misc: ' + str(row.Other),
                                                         'Date: ' + row.Date
                                                        ]
    except:
        pass
    
    if len(locate_box.keys()) > 0:
        return (locate_box)

Get locations and information for matching primers in each sequence.

In [12]:
all_matches = []
for sequence in sequence_list:
    df['fwd_match'] = df.apply(locateFprimer, axis=1)
    df['rev_match'] = df.apply(locateRprimer, axis=1)

    print ("For sequence", sequence[:7] + '...' + sequence[-7:], "there are", df.fwd_match.count(), "matching forward primers", 'and', df.rev_match.count(), 'matching reverse primers.')
    print ()
    matches_per_seq_F = df.fwd_match.dropna()
    matches_per_seq_R = df.rev_match.dropna()
    all_matches.append([matches_per_seq_F, matches_per_seq_R])

For sequence agct...agct there are 0 matching forward primers and 0 matching reverse primers.



In [13]:
all_matches

[[Series([], Name: fwd_match, dtype: object),
  Series([], Name: rev_match, dtype: object)]]

In [14]:
def matches(match):
    match_dict = {}
    for i in match:
        if list(i.keys())[0] not in match_dict.keys():
            match_dict[list(i.keys())[0]] = [list(i.values())[0]]
        else:
            match_dict[list(i.keys())[0]].append(list(i.values())[0])
    return match_dict

for sequence_match in all_matches:
    fwd_matches = sequence_match[0].tolist()
    rev_matches = sequence_match[1].tolist()
    
    fwd_dict = matches(fwd_matches)
    rev_dict = matches(rev_matches)

print (sorted(fwd_dict.keys()), sorted(rev_dict.keys()))

[] []


In [15]:
print('_'*8+'a'+'_')
print(sequence[:10])
print ('at'+'_'*8)

________a_
agct
at________


In [16]:
pri = 'atttaactttgacgg'
pri in sequence
ind = sequence.find(pri)
sequence[ind:ind+15]

't'

In [17]:
new_seq = sequence[:ind] + '___' + sequence[ind:ind+15] + '___' + sequence[ind+15:]

In [18]:
prime = 'aaaattttaaaccc'
prime[::-1]

'cccaaattttaaaa'